### Holis!
Intentemos recrear nuestro sistema de recolección. Primero, definamos lo que queremos hacer:

_Un sistema que, una vez ejecutado el archivo, descargue todas las imágenes de las cámaras que nos interesan, las clasifique, y guarde los resultados._

Suena complicado. Ahora, lo importante es dividir la tarea en pedacitos manejables:

* Un sistema => Un archivo
    * Descargar las imágenes
         * Tener todas las cámaras en un diccionario cámara -> url de youtube
         * Obtener la información de cada video, obteniendo un url de stream!
            * yt-dlp tiene una función que te permite hacer eso
         * Descargar el último frame del stream de cada cámara
            * CV2 tiene una clase que te permite abrir un stream, pasar al último frame, y guardarlo como imágen
         * Guardar cada imagen en disco
            * Decide un formato de guardado y usa el sistema de archivos.
         * También guardar el tiempo de recolección o no nos sirve de nada! 
    * Clasificar las imágenes
         * Cargar el modelo UNA VEZ! (y el procesador de imágenes)
         * Tomar cada frame (ver procedimiento anterior)
         * Pasar cada frame al modelo -> te dará outputs
         * Pasar los outputs a probabilidades (usar softmax?)
    * Guardar los resultados
         * Crear un dataframe (Path, cámara, tiempo probabilidades
         * Cargar el dataframe con toda la información
             * Si encuentras el archivo principal y se abre, ese lo cargas
             * Si no, busca el archivo secundario y si abre, lo cargas
             * Y si no, haces uno vacío.
         * Juntar ambos dataframes
         * Guardar el dataframe compuesto
             * Antes de siquiera tocar el dataframe grande, guardas el backup en el archivo secundario. Si eso falla, aborta todo.
          

### LIBRERÍAS
Cool! Ahora tenemos que empezar. Primero, es buena práctica cargar *todas* las librerías que usas en un archivo al principio, así que haz eso. Si te falta una ntp, vuelves y la pones.

### Descarga de imágenes
Ahora ya sabes que hacer aquí, te diré cómo espero que quede. Quiero:
* Una función, save_frame(frame, time, camera) que te guarde un frame generando el path en base al tiempo y la cámara.
* Una función, download_last_frame(stream_url) que, dado un string que sea un url a un stream (no al video en youtube directamente!) te descargue el último frame del stream y el tiempo en UTC en el que descargó (esto lo obtienes con datetime en vez del stream, probablemente).
* Una función, get_stream(url) que, dado un url de youtube, te de un url que represente un stream de video.
* Un diccionario que tenga todas las cámaras y sus respectivos urls de youtube.

Te das cuenta que hice lo mismo que antes pero al revés y ahora todo encaja junto? El diseño puede estar difícil, pero ya diseccionado debería solo organizar el código que ya sabes. Dale.

### Clasificación
Ya me leíste, cargas el modelo **una vez** (cada vez que lo cargas tarda bastante, no quiero repetir esto así que apréndetelo, se cargan los datos la menor cantidad de veces posible en casi todos los casos), y me das lo siguiente:
* Una función, classify_frame(frame), que te devuelva las probabilidades de cada clase en una serie de pandas (con el nombre adecuado de cada clase), así como la clase principal

Fácil, no? Aquí lo dificil es la implementación en vez del diseño, así que te diré cómo cargar el modelo bien. Para empezar, tu modelo está en el siguiente path: `A:\\MimModels\\swinv2-base-patch4-window16-256-popo\\` (Por si te lo preguntas, Linux usa `/`, Windows usa `\\` y permite `/`, pero no se prefiere.)

Nuestro modelo se compone de dos partes, el preprocesador de imágenes (reescala y eso tus imágenes para que le gusten al modelo), y el modelo en sí (que aprendió a clasificar volcanes). Cargas ambos pasándole la ubicación del modelo a las clases correspondientes (AutoImageProcessor, AutoModelForImageClassification) y transformers te lo carga solito porque el modelo le dice qué clase de modelo es. Una vez cargado, si quieres clasificar, debes hacer lo siguiente:
* input frame -> processed_frame [image_processor(images=frame, return_tensors="pt")["pixel_values"\]
* processed_frame -> outputs [model(processed_frame).logits\]
* outputs -> probabilities [softmax\]
* probabilities -> class_index [probabilities.argmax(dim=-1)\]
* class_index -> class_label [model.config.id2label[class_idx\]\] (El modelo se aprendió los nombres, te da id2label y label2id como diccionarios.)

### Guardar los datos
Irónicamente, esto podría ser la parte teóricamente más complicada de esto. Quiero que no solo abras y cierres un dataframe, quiero que lo hagas de forma segura: si un guardado falla y te corrompe tus datos, pierdes 24+ horas de puro trabajo, y eso está súper no chido porque nada de lo que hiciste antes sobrevive. Así se guarda algo de forma un poco más segura:

* Cargas lo que necesitas (más sobre esto ahorita)
* Lo guardas de **inmediato** como *backup*
* Modificas la versión que ya cargaste
* Lo guardas como *el principal*

Ahora si algo falla en el punto de modificar los datos, o en guardarlos, tienes un backup que está OK. El problema es que cargar los datos ahora no es tan sencillo:
* *Intentas* cargar el principal
* Si jala, ya terminaste, si no, intentas cargar el backup.
* Si jala, ya terminaste, si no, *creas datos vacíos* que puedan usarse como si tuvieran cosas.
Bajo este principio, nunca terminas sin cargar, aunque posiblemente cargues nada. Hay diferencia entre vacío y nulo, ver el siguiente diagrama:

<img src="https://th.bing.com/th/id/OIP.Zckuards55bS9zQ6KzJBhQAAAA?rs=1&pid=ImgDetMain"/>

En python, undefined te daría un error si siquiera preguntaras si existe (es usar variables que no declaraste), null te daría un error si intentaras usarlo (el clásico **None**), pero puedes trabajar con "0", en nuestro caso un dataframe vacío.

Quiero:
* Una función, load_data(), que te de los datos guardados más actuales de forma segura siguiendo nuestros pasos. **Esta función se debe encargar del backup!!!**
* Una función, save_data(df), que te guarde los datos *solo en el archivo principal*.

### Finalizando todo

Genial, si ya llegaste aquí, te falta solo una cosa muuuuy importante: usar todo lo que acabas de definir. Me haces:
* Una función, classify_all(), que te descargue los frames de todas las cámaras del diccionario de cámaras, y te devuelva una lista de series de pandas de los frames clasificados con classify_frame(frame)

Y la usas para obtener la lista de series, la concatenas para tener un dataframe, cargas los datos anteriores con load_data(), concatenas los datos con tus nuevos datos, y luego usas save_data(df).
Y ya, quedó el sistema. Bien hecho en llegar aquí!!!, ahora sólo termina el trabajo.